In [1]:
import numpy as np
import pandas as pd

In [2]:
employ = pd.read_excel('附件3\操作人员信息表.xlsx')
employ.head()

操作人员编号  工龄 生产线编号
0   A001   1  M301
1   A002   5  M302
2   A003   6  M303
3   A004   4  M304
4   A005   2  M305

In [3]:
employee_count , intense =employ.shape
employee_count

10

In [4]:
age_1=0
age_2=0
age_3=0
age_4=0
age_5=0
age_6=0
for i in range(employee_count):
    if employ['工龄'][i]==1:
        age_1+=1
    elif employ['工龄'][i]==2:
        age_2+=1
    elif employ['工龄'][i]==3:
        age_3+=1
    elif employ['工龄'][i]==4:
        age_4+=1
    elif employ['工龄'][i]==5:
        age_5+=1
    elif employ['工龄'][i]==6:
        age_6+=1
AGE_GROUPS={
    '1':age_1/employee_count,
    '2':age_2/employee_count,
    '3':age_3/employee_count,
    '4':age_4/employee_count,
    '5':age_5/employee_count,
    '6':age_6/employee_count
}

In [5]:
AGE_GROUPS

{'1': 0.2, '2': 0.2, '3': 0.1, '4': 0.2, '5': 0.2, '6': 0.1}

In [6]:
all_time = 10*3*7*8 # 工厂每周的工厂小时
one_person_time = 5*8 # 个人的一周工作时长
need_employee=int(all_time/one_person_time)
need_employee

42

In [7]:
import pandas as pd

need = [8, 9, 4, 8, 9, 4]
# temp = pd.DataFrame(columns=['操作人员编号', '工龄'])
temp=[]
k = 1

for i in range(len(need)):
    for j in range(need[i]):
        row_data = {'操作人员编号': f'B{k:03d}', '工龄': i+1}
        # row_data=pd.DataFrame(row_data)
        temp.append(row_data)
        k += 1

temp_day=[]
for i in range(1,366):
    temp_day.append({'日期':str(i)})
temp_day=pd.DataFrame(temp_day)
temp = pd.DataFrame(temp)
temp.head()

操作人员编号  工龄
0   B001   1
1   B002   1
2   B003   1
3   B004   1
4   B005   1

In [8]:
from pyscipopt import Model, quicksum

model = Model("scheduling")

# Creating variables
x = {}
for i in range(need_employee):
    for j in range(7):
        for k in range(3):
            x[i, j, k] = model.addVar(vtype="B", name="x_%d_%d_%d" % (i, j, k))
# Adding constraints
for i in range(need_employee):
    for j in range(7):
        expr = quicksum(x[i, j, k] for k in range(3)) <= 1
        constraint = model.addCons(expr, "每天只上一次班")

for i in range(need_employee):
    expr = quicksum(x[i, j, 0]  for j in range(7)) >= 1
    constraint = model.addCons(expr, "每周起码上一次早班")

    expr = quicksum(x[i, j, 1]  for j in range(7)) >= 1
    constraint = model.addCons(expr, "每周起码上一次午班")

    expr = quicksum(x[i, j, 2]  for j in range(7)) >= 1
    constraint = model.addCons(expr, "每周起码上一次晚班")

for i in range(need_employee):
    for j in range(7-1):
        expr = x[i, j, 2] + x[i, j + 1 , 0] <= 1
        constraint = model.addCons(expr, "晚班和第二天早班不能连着上")

for i in range(need_employee):
    expr = quicksum(x[i, j, k] for j in range(7) for k in range(3)) == 5
    constraint = model.addCons(expr, "每周需要上5天班")


for j in range(7):
    expr = quicksum(x[i, j, k]for i in range(need_employee) for k in range(0,1)) == 10
    constraint = model.addCons(expr, "每次早班需要有10个人") 
    
    expr = quicksum(x[i, j, k]for i in range(need_employee) for k in range(1,2)) == 10
    constraint = model.addCons(expr, "每次午班需要有10个人") 
    
    expr = quicksum(x[i, j, k]for i in range(need_employee) for k in range(2,3)) == 10
    constraint = model.addCons(expr, "每次晚班需要有10个人") 

# Setting the objective function
objective = quicksum(x[i, j, k] for i in range(need_employee) for j in range(7) for k in range(3))
model.setObjective(objective, "maximize")

# Solving the model
model.optimize()


In [9]:
# 在 model.optimize() 之后添加以下代码
if model.getStatus() == "optimal":
    print("ok!!!")
else:
    print("模型没有找到最优解")


ok!!!


In [10]:
import dask.dataframe as dd
import dask.multiprocessing as mp

In [11]:
data = dd.read_csv('附件3/M302.csv')

In [12]:
import pandas as pd

# 假设model.getVal()函数已经定义好了，并且能够返回正确的值

# 初始化一个空的DataFrame，用于存储员工的班次信息
person = pd.DataFrame(index=range(365), columns=range(42))

# 循环遍历每一天
for i in range(365):
    # 循环遍历每一名员工
        # 初始化员工当天的班次列表
    shifts = []
        # 循环遍历一周中的每一天（0-6）
    for j in range(42):
            # 循环遍历三个班次
        flag = 0
        for k in range(3):
            # 获取当前班次的值
            value = model.getVal(x[j, i%7, k])
            # 根据班次类型添加到员工班次列表中
            if value == 1:
                if k == 0:
                    person.at[i, j] = '早'
                    flag = 1
                elif k == 1:
                    person.at[i, j] = '中'
                    flag = 1
                elif k == 2:
                    person.at[i, j] = '晚'
                    flag = 1
    # 将员工当天的班次信息添加到DataFrame中
        if flag == 0:
            person.at[i, j] = '休'
        
# 现在person就是您想要的格式，365行（天数）和42列（员工），每行包含当天每位员工的班次信息

In [13]:
person

0  1  2  3  4  5  6  7  8  9   ... 32 33 34 35 36 37 38 39 40 41
0    早  晚  休  休  休  休  休  中  休  休  ...  早  早  晚  晚  晚  早  早  休  晚  休
1    中  休  早  休  早  早  休  休  早  早  ...  晚  休  中  中  休  晚  中  中  晚  休
2    晚  休  中  早  休  早  早  休  休  早  ...  晚  晚  中  晚  晚  中  休  早  中  晚
3    休  中  早  中  中  中  早  早  早  早  ...  休  休  早  休  晚  晚  中  中  休  晚
4    晚  晚  早  早  早  早  早  早  早  休  ...  中  中  休  休  休  休  晚  晚  早  中
..  .. .. .. .. .. .. .. .. .. ..  ... .. .. .. .. .. .. .. .. .. ..
360  休  中  早  中  中  中  早  早  早  早  ...  休  休  早  休  晚  晚  中  中  休  晚
361  晚  晚  早  早  早  早  早  早  早  休  ...  中  中  休  休  休  休  晚  晚  早  中
362  晚  中  晚  晚  中  晚  晚  晚  晚  中  ...  休  中  早  早  早  休  中  休  休  早
363  休  早  休  中  晚  休  中  中  中  晚  ...  晚  中  休  中  中  晚  休  早  晚  中
364  早  晚  休  休  休  休  休  中  休  休  ...  早  早  晚  晚  晚  早  早  休  晚  休

[365 rows x 42 columns]

In [14]:
person.columns=temp['操作人员编号']
person.index=temp_day['日期']

In [15]:
person.head()

操作人员编号 B001 B002 B003 B004 B005 B006 B007 B008 B009 B010  ... B033 B034 B035  \
日期                                                        ...                  
1         早    晚    休    休    休    休    休    中    休    休  ...    早    早    晚   
2         中    休    早    休    早    早    休    休    早    早  ...    晚    休    中   
3         晚    休    中    早    休    早    早    休    休    早  ...    晚    晚    中   
4         休    中    早    中    中    中    早    早    早    早  ...    休    休    早   
5         晚    晚    早    早    早    早    早    早    早    休  ...    中    中    休   

操作人员编号 B036 B037 B038 B039 B040 B041 B042  
日期                                         
1         晚    晚    早    早    休    晚    休  
2         中    休    晚    中    中    晚    休  
3         晚    晚    中    休    早    中    晚  
4         休    晚    晚    中    中    休    晚  
5         休    休    休    晚    晚    早    中  

[5 rows x 42 columns]

In [16]:
person.to_excel("附件4/result4-1.xlsx")

In [17]:
temp_day=[]
temp_day_time=[]
for i in range(1,366):
    for j in range(0,3):
        if j==0:
            temp_day.append({'日期':str(i)})
            temp_day_time.append({'班次':'早'})
        elif j==1:
            temp_day.append({'日期':str(i)})
            temp_day_time.append({'班次':'中'})
        elif j==2:
            temp_day.append({'日期':str(i)})
            temp_day_time.append({'班次':'晚'})
temp_day=pd.DataFrame(temp_day)
temp_day_time = pd.DataFrame(temp_day_time)

In [18]:
factory = pd.read_excel('附件4/result4-2.xlsx')

In [19]:
factory = factory.columns

In [20]:
ans = pd.DataFrame(index=temp_day['日期'], columns=factory[1::])
x, y = ans.shape

for i in range(x):
    # 计算索引i对应的时间段（早、午、晚）
    period = '早' if i % 3 == 0 else ('中' if i % 3 == 1 else '晚')
    # 在第0列（即日期列）填入对应的时间段
    ans.iat[i, 0] = period
for i in range(365*3):
    j=1
    for k in range(42):
        if person.iloc[int(i/3),k]==ans.iloc[i,0]:
            ans.iat[i,j]=person.columns[k]
            j+=1
            if j==11:break

In [21]:
ans

班次  M301  M302  M303  M304  M305  M306  M307  M308  M309  M310
日期                                                                
1    早  B001  B019  B025  B028  B029  B030  B033  B034  B038  B039
1    中  B008  B015  B017  B018  B021  B022  B023  B024  B026  B031
1    晚  B002  B012  B013  B016  B020  B032  B035  B036  B037  B041
2    早  B003  B005  B006  B009  B010  B014  B015  B022  B023  B031
2    中  B001  B012  B017  B018  B021  B024  B035  B036  B039  B040
..  ..   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
364  中  B004  B007  B008  B009  B011  B013  B034  B036  B037  B042
364  晚  B005  B010  B012  B015  B018  B027  B032  B033  B038  B041
365  早  B001  B019  B025  B028  B029  B030  B033  B034  B038  B039
365  中  B008  B015  B017  B018  B021  B022  B023  B024  B026  B031
365  晚  B002  B012  B013  B016  B020  B032  B035  B036  B037  B041

[1095 rows x 11 columns]

In [23]:
ans.to_excel("附件4/result4-2.xlsx")